<a href="https://colab.research.google.com/github/comet-toolkit/comet_training/blob/main/obsarray_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install obsarray

In [ ]:
import xarray as xr
import obsarray

In [ ]:
# Read Dataset contents

ds = xr.open_dataset("obs_example.nc")

In [ ]:
# Inspect Dataset contents

print("Observation", ds.unc.obs_vars)   # Variables with uncertainties
print("Uncertainty", ds.unc.unc_vars)   # Uncertainty variables

In [ ]:
# Inspect uncertainty variables for a particular variable

print(ds.unc["temperature"])

In [ ]:
# Add uncertainty variable 

# Define uncertainty variable as tuple: (dims, data, attrs) 
# where attrs should include:
# > "err_corr" - error-correlation structure defintion (if omitted assumed random)
# > "pdf_shape" - uncertainty pdf shape (if omitted assumed gaussian)

ds.unc["temperature"]["u_sys_temperature"] = (
    ["x", "y", "time"],
    ds.temperature * 0.03,
	{
        "err_corr": [
            {
                "dim": "x",
                "form": "systematic",
                "params": [],
            },
            {
                "dim": "y",
                "form": "systematic",
                "params": [],
            },
            {
                "dim": "time",
                "form": "systematic",
                "params": [],
            }
        ]
    },
)

# Now inspect again

print(ds.unc["temperature"])



In [ ]:
# Get total variable uncertainty

ds.unc["temperature"].total

In [ ]:
# Get total variable random uncertainty

ds.unc["temperature"].random

In [ ]:
# Get total variable systematic uncertainty

ds.unc["temperature"].systematic

In [ ]:
# Inspect uncertainty variables
for unc_var in ds.unc["temperature"]:
    print(unc_var, "\n")

In [ ]:
# Test if uncertainty fully random in all dimensions
print("u_ran_temperature random: ", ds.unc["temperature"]["u_ran_temperature"].is_random)
print("u_str_temperature random: ", ds.unc["temperature"]["u_str_temperature"].is_random)
print("u_sys_temperature random: ", ds.unc["temperature"]["u_sys_temperature"].is_random)

# Test if uncertainty fully systematic in all dimensions
print("u_ran_temperature systematic: ", ds.unc["temperature"]["u_ran_temperature"].is_systematic)
print("u_str_temperature random: ", ds.unc["temperature"]["u_str_temperature"].is_systematic)
print("u_sys_temperature systematic: ", ds.unc["temperature"]["u_sys_temperature"].is_systematic)

In [ ]:
# Get uncertainty info for given variable

# data (return data array)
print("Data:\n", ds.unc["temperature"]["u_str_temperature"].data)

# pdf shape
print("\nPDF Shape:\n", ds.unc["temperature"]["u_str_temperature"].pdf_shape)

# for error-correlation, error-correlation interface object returned
print("\nError Correlation:\n", ds.unc["temperature"]["u_str_temperature"].err_corr)

In [ ]:
# Inspect error-correlation information
ds.unc["temperature"]["u_str_temperature"].err_corr.params

In [ ]:
# Inspect error-correlation information for a particular dimension
ds.unc["temperature"]["u_str_temperature"].err_corr.dim_params("x")

In [ ]:
# Build error-correlation matrix (ordered as ds.variable.flatten())
ds.unc["temperature"]["u_ran_temperature"].err_corr.to_matrix()

In [ ]:
# Build error-correlation matrix or for one dimension
# (or set of dimensions if error-correlation defintion spans multiple dimensions)

ds.unc["temperature"]["u_str_temperature"].err_corr.to_matrix(dim="x")